Sensor Fusion Function (sensor_fusion)

This function implements a particle filter for sensor fusion. It takes the following arguments:

particles: Array of particles representing possible states (e.g., positions).

weights: Array of weights associated with each particle.

velocity: Motion vector representing velocity.

measurements: Sensor measurements (e.g., positions of objects).

sensor_noise (optional): Standard deviation of sensor noise (defaults to 0.1).

The function performs the following steps:

Prediction Step (Motion Model): Updates the particle positions based on the predicted movement using the velocity vector.
Update Step (Sensor Measurements): Iterates through each particle and calculates the distance between the particle and each sensor measurement. Adds sensor noise to simulate realistic sensor readings. Assigns a higher weight to particles closer to the sensor measurements using a Gaussian likelihood function.

Normalization Step: Normalizes the weights to sum to 1.

Resampling Step (resample_particles)

This function resamples particles based on their weights. Particles with higher weights are more likely to be selected. This step helps to maintain diversity among particles and avoids getting stuck in local minima.

Imagine searching for a friend in a crowded park. You have some initial guesses about where they might be (particles). If you hear they are walking towards the ice cream stand (velocity), you wouldn't keep searching in the same spot. You would update your search area based on their predicted movement (similar to updating particle positions). This helps the filter focus its search on more likely locations as time progresses.

 By adding the velocity (predicted movement) to each particle's location, you essentially predict where each possible location (particle) might move in the next time step. This refines the distribution of particles to be more concentrated around areas where the human is likely to be based on their past movement.

- Sensor Data: A camera captures video footage of an environment where a person might be present.

- Data Preprocessing: The video frames might undergo preprocessing to isolate potential human features (shapes, colors) from the background.

- Particle Filter:

The filter initializes a number of particles (representing possible locations of the person) scattered around the area of interest.
Each particle has an associated weight, indicating its likelihood of being the actual person's location.
The filter uses a motion model to predict how the person might move (considering walking speed, direction changes). It updates particle positions based on this model.
Sensor information (preprocessed data) is used to compare predicted locations with what the camera actually sees. Particles closer to detected features receive higher weights.
Weight Update and Resampling: Particles with higher weights are more likely to represent the true location. The filter resamples the particle set, creating more copies of high-weight particles and discarding those with low weights. This concentrates the particle distribution around more probable locations.

- Output: The particle with the highest weight (or the average position of top-weighted particles) is considered the estimated location of the person within the image/environment

In [ ]:
import numpy as np

In [ ]:
def sensor_fusion(particles, weights, velocity, measurement, sensor_noise=0.1):
    """
    Implements sensor fusion using a particle filter for a single observation.

    Args:
        particles (np.ndarray): Array of particles representing possible states (e.g., positions).
        weights (np.ndarray): Array of weights associated with each particle.
        velocity (np.ndarray): Motion vector representing velocity for the current observation.
        measurement (np.ndarray): Sensor measurement (e.g., position of the current observation).
        sensor_noise (float, optional): Standard deviation of sensor noise (defaults to 0.1).

    Returns:
        tuple: Updated particles and weights after sensor fusion for the current observation.
    """
    # Prediction Step (Motion Model)
    particles += velocity  # Update particle positions based on predicted movement

    # Update Step (Sensor Measurement)
    for i, particle in enumerate(particles):
        particle_measurement_dist = np.linalg.norm(particle - measurement)
        # Add sensor noise to simulate realistic sensor readings
        particle_measurement_dist += np.random.normal(scale=sensor_noise)
        # Likelihood based on distance (closer = higher likelihood)
        weights[i] *= np.exp(-(particle_measurement_dist**2) / (2 * sensor_noise**2))

    # Normalize weights to sum to 1
    weights /= np.sum(weights)

    return particles, weights

# Resampling Step
def resample_particles(particles, weights):
    """
    Resamples particles based on their weights.

    Args:
        particles (np.ndarray): Array of particles.
        weights (np.ndarray): Array of weights associated with each particle.

    Returns:
        np.ndarray: Resampled particles.
    """

    indices = np.random.choice(range(len(particles)), len(particles), p=weights)
    return particles[indices]
# Example Usage
# Assuming 2D positions (x, y)
num_particles = 100
particles = np.random.rand(num_particles, 2)
weights = np.ones(num_particles) / num_particles
estimated_position=[]
velocities = [np.array([0.1, 0.2]), np.array([0.2, 0.3]), np.array([0.3, 0.4])]  # Example velocities for each observation
measurements = [np.array([1.0, 2.0]), np.array([2.0, 3.0]), np.array([3.0, 4.0])]  # Example sensor measurements for each observation

# Sensor Fusion Loop
for velocity, measurement in zip(velocities, measurements):  # Iterate over each observation
    particles, weights = sensor_fusion(particles, weights, velocity, measurement)

    # Optional: Resampling (can be done periodically)
    particles = resample_particles(particles, weights)

    # Access estimated position (e.g., weighted mean)
    estimated_position.append(np.average(particles, weights=weights, axis=0))

print("Estimated Position:", estimated_position)


Estimated Position: [array([1.05934779, 1.05562396]), array([1.25952281, 1.35555073]), array([1.55952281, 1.75555073])]
